In [55]:
from comppy.clt import Layup
from comppy.clt import Laminate
from comppy.clt import Material
from comppy.clt import MatLib
from comppy.clt import Laminate
from comppy.clt import Requirements
import numpy as np

## Defining laminates manually
### Begin by defining the ply orientation and material layups
The layup orientation string should match the layup material string. The standard ASTM D6507 orientation codes are also valid for defining the material order.

In [56]:
#e.g., layup_orientations = "[+-45/(45/90):s/0:2]:s"
#                         = "[+-45/(45/90):s/0\]:s"
layup_orientations = "[90/0/45/0]"
layup_materials = "[material1/material1/material1/material1]"

Generate a Layup object from the layup orientation and material strings.

In [57]:
layup_1 = Layup.gen_from_short(layup_orientations, layup_materials)

Inspect the layup orientation and material lists:

In [58]:
print(layup_1.layup_list)
print(layup_1.material_list)

[90.0, 0.0, 45.0, 0.0]
['material1', 'material1', 'material1', 'material1']


### Define materials and build a material library
The name property of Material objects should match the material names used in the layup material string.

In [59]:
#material = Material(name, ex, ey, gxy, vxy, t) or material = Material.gen_from_dict(dict_of_material_properties)
m1 = Material('material1', 70, 70, 35, 0.13, 0.1)
m2 = Material('material2', 140, 25, 35, 0.13, 0.15)
m3 = Material('material3', 73, 73, 35, 0.13, 0.2)

MatLib objects can be defined from Material objects or a dictionary of material properties.

In [60]:
# library = MatLib(material1, material2, material3,...) or library=MatLib(dict_of_dict_of_material_definitions)
# Using material objects
library1 = MatLib(m1, m2, m3)

### Define a Laminate object from the Layup and MatLib objects

In [61]:
laminate1 = Laminate(layup_1, library1)

### Inspect the properties of the Laminate
The ABD matrix is 6x6 numpy array

In [76]:
print(np.array2string(laminate1.abd, max_line_width=np.inf))
print(laminate1.validate(laminate1_reqs))

[[ 2.88839894e+01  3.29991862e+00 -2.71069312e-07  2.01327434e-02 -2.01327434e-02  9.48742591e-08]
 [ 3.29991862e+00  2.88839894e+01  2.71069312e-07 -2.01327434e-02  2.01327434e-02 -9.48742591e-08]
 [-2.71069312e-07  2.71069312e-07  1.35973451e+01  9.48742591e-08 -9.48742591e-08 -2.01327434e-02]
 [ 2.01327434e-02 -2.01327434e-02  9.48742591e-08  3.81093310e-01  4.80254637e-02 -1.17463368e-08]
 [-2.01327434e-02  2.01327434e-02 -9.48742591e-08  4.80254637e-02  3.81093310e-01  1.17463368e-08]
 [ 9.48742591e-08 -9.48742591e-08 -2.01327434e-02 -1.17463368e-08  1.17463368e-08  1.85324484e-01]]
{'symmetric': False, 'balanced': True, 'flexurally balanced': True}


Inspect the effective elastic constants:

In [63]:
for prop_type, properties in laminate1.effective_props.items():
    print(prop_type, 'properties:')
    for prop_name, prop in properties.items():
        print(prop_name,":", prop)

unrestrained properties:
e_x : 71.25990118745815
e_y : 71.25990118745813
v_xy : 0.11434122809873463
v_yx : 0.11434122809873462
g_xy : 33.9878950255119


### Generate a Requirements object to specify design requirements
You can specify the numerical tolerance of the symmetry, balanced and flexurally balanced conditions

In [64]:
laminate1_reqs = Requirements(symmetry=1e-6, balanced=1e-6, flex_bal=1e-6)

Test the Laminate object to see if it meets design requirements:

In [65]:
dict_of_conditions = laminate1.validate(laminate1_reqs)

See if the Laminate meets design requirements:

In [66]:
for condition, validity in dict_of_conditions.items():
    print('Was the laminate', condition+'?', validity)

Was the laminate symmetric? False
Was the laminate balanced? True
Was the laminate flexurally balanced? True


## Generate a Laminate from design requirements
### Begin by defining a MatLib object and a Requirements object

In [67]:
# Using a dictionary of material properties
library2 = MatLib({'material1': {'e1': 90, 'e2': 15, 'g12': 35, 'v12': 0.013, 't': 0.1},'material2': {'e1': 70, 'e2': 70, 'g12': 35, 'v12': 0.013, 't': 0.1}} )
# Requirements object
laminate2_reqs = Requirements(symmetry=1e-6, balanced=1e-6, flex_bal=1e-6)

### Generate a Laminate object by submitting the MatLib and Requirements objects to Laminate.gen_from_reqs

In [68]:
laminate2 = Laminate.gen_bal_sym(library2, laminate2_reqs)

In [87]:
Laminate.return_q_bar(library1.material1, -45)

array([[ 7.52298851e+01,  5.22988506e+00, -2.71069311e-06],
       [ 5.22988506e+00,  7.52298851e+01,  2.71069311e-06],
       [-2.71069312e-06,  2.71069312e-06,  3.09734513e+01]])

In [82]:
library1.material1.q

array([[71.20333638,  9.25643373,  0.        ],
       [ 9.25643373, 71.20333638,  0.        ],
       [ 0.        ,  0.        , 35.        ]])